# Create local Neo4j Knowledge Graph (for testing)
This notebook is provided to test creating a knowledge graph using a locally installed version of Neo4j.

**Note, make sure that the Neo4j Browser has been shutdown before running this notebook.**

The path `NEO4J_HOME` must be set to the Neo4j installation directory path.

This notebook creates a local version of the knowledge graph by ingesting the .csv files in the Neo4j import directory `NEO4J_HOME/import`.

Note: This notebook works only on Linux and Mac OS.



In [1]:
import os
import time
from pathlib import Path

In [2]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0


### Start Neo4j database

**Note, make sure that the Neo4j Browser has been shutdown before running this notebook.**

In [3]:
status = !"$NEO4J_HOME"/bin/neo4j status
status = str(status)
while not 'Neo4j is running' in status:
    !"$NEO4J_HOME"/bin/neo4j start
    time.sleep(30)
    status = !"$NEO4J_HOME"/bin/neo4j status
    status = str(status)
    print(status)

# wait until neo4j is ready to receive requests
time.sleep(60)

Directories in use:
  home:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0
  config:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/conf
  logs:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/logs
  plugins:      /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/plugins
  import:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/import
  data:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d

### Run Cypher commands to create Knowledge Graph

In [4]:
start = time.time()

In [5]:
# TODO create a batch script for Windows
!../../scripts/run_cyphers.sh -init # start with a clean database and create new indices and constraints
#!../../scripts/run_cyphers.sh

Endpoint: bolt://localhost:7687
Username: neo4j
Password: neo4jbinder
 
----------------------------------------------
Running 00a-Init.cypher:
 
// delete all nodes
//MATCH (n) DETACH DELETE n; # deleting all nodes at once leads to out of memory error
// Delete in small batches
call apoc.periodic.iterate("MATCH (n) return n", "DETACH DELETE n", {batchSize:1000}) yield batches, total 
RETURN batches, total;

// delete all constraints and indices
CALL db.index.fulltext.drop('locations');
CALL db.index.fulltext.drop('bioentities');
CALL db.index.fulltext.drop('geoids');
CALL apoc.schema.assert({},{}, true);
                                                                                            
// create constraints and indices
CREATE CONSTRAINT location ON (n:Location) ASSERT n.id IS UNIQUE;
CREATE INDEX location_n FOR (n:Location) ON (n.name);
CREATE INDEX location_l FOR (n:Location) ON (n.location);
CREATE CONSTRAINT world ON (n:World) ASSERT n.id IS UNIQUE;
CREATE CONSTRAINT unre

In [6]:
end = time.time()
print('time:', end-start)

time: 1196.8313286304474


### Make sure to shutdown Neo4j!

In [7]:
!"$NEO4J_HOME"/bin/neo4j stop

Stopping Neo4j........ stopped
